In [36]:
import tfim
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy import optimize
import progressbar

In [37]:
PBC = True
L = [3]
J = 1

In [38]:
# Build lattice and basis
###################################
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)
###################################

In [39]:
seed_range = range(10);

Jij_array = [];

for i in seed_range:
    Jij = tfim.Jij_instance(N,J,"bimodal",i) 
    Jij_array.append(Jij)

In [40]:
# Calculate energy array:

indices_array = []

for Jij in Jij_array:
    Energies = tfim.JZZ_SK_ME(basis,Jij) # Why was there a minus sign?
    GS_energy = np.min(Energies)
    GS_indices = np.nonzero(Energies == GS_energy)[0]
    indices_array.append(GS_indices)

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|


In [41]:
# Define Hamming distance
def Hamming_distance(state_1, state_2):
    return len(np.nonzero(state_1 - state_2)[0])

# Calculate Hamming distance array

def Hamming_array(GS_indices):
    Hamming = np.zeros((len(GS_indices), len(GS_indices)-1))
    for i, n in enumerate(GS_indices):
        basis_n = basis.state(n)
        for j, m in enumerate(np.delete(GS_indices,i)):
            basis_m = basis.state(m)
            Hamming[i, j] = Hamming_distance(basis_n, basis_m)
    return Hamming

In [42]:
def search_2(array):
    if np.amin(array) == 2:
        return True
    else:
        return False

In [45]:
# search for Hamming distance 2

seed_list = []

for index, indices in enumerate(indices_array):
    if search_2(Hamming_array(indices)):
        seed_list.append(index)
    print Hamming_array(indices)
        
print seed_list

[[1. 2. 1. 2. 3.]
 [1. 1. 2. 3. 2.]
 [2. 1. 3. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 3. 2. 1. 1.]
 [3. 2. 1. 2. 1.]]
[[1. 2. 1. 2. 3.]
 [1. 1. 2. 3. 2.]
 [2. 1. 3. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 3. 2. 1. 1.]
 [3. 2. 1. 2. 1.]]
[[1. 2. 1. 2. 3.]
 [1. 1. 2. 3. 2.]
 [2. 1. 3. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 3. 2. 1. 1.]
 [3. 2. 1. 2. 1.]]
[[1. 1. 2. 2. 3.]
 [1. 2. 1. 3. 2.]
 [1. 2. 3. 1. 2.]
 [2. 1. 3. 2. 1.]
 [2. 3. 1. 2. 1.]
 [3. 2. 2. 1. 1.]]
[[1. 1. 2. 2. 3.]
 [1. 2. 1. 3. 2.]
 [1. 2. 3. 1. 2.]
 [2. 1. 3. 2. 1.]
 [2. 3. 1. 2. 1.]
 [3. 2. 2. 1. 1.]]
[[1. 1. 2. 2. 3.]
 [1. 2. 1. 3. 2.]
 [1. 2. 3. 1. 2.]
 [2. 1. 3. 2. 1.]
 [2. 3. 1. 2. 1.]
 [3. 2. 2. 1. 1.]]
[[1. 1. 2. 2. 3.]
 [1. 2. 1. 3. 2.]
 [1. 2. 3. 1. 2.]
 [2. 1. 3. 2. 1.]
 [2. 3. 1. 2. 1.]
 [3. 2. 2. 1. 1.]]
[[1. 2. 1. 2. 3.]
 [1. 1. 2. 3. 2.]
 [2. 1. 3. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 3. 2. 1. 1.]
 [3. 2. 1. 2. 1.]]
[[1. 2. 1. 2. 3.]
 [1. 1. 2. 3. 2.]
 [2. 1. 3. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 3. 2. 1. 1.]
 [3. 2. 1. 2. 1.]]
[[1. 1. 2. 2. 3.]
 

In [44]:
# pass Jij, GS_states only, build each order of perturbation theory one at a time
# module: 1st order, and second order matrices